In [20]:
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

In [21]:
df = pd.read_csv(r"C:\Users\wartm\Documents\FHNW\immo_challenge\data\immo_data_202208_v2.csv")

C:\Users\wartm\AppData\Local\Temp\ipykernel_16252\1417368699.py:1: DtypeWarning: Columns (3,4,5,6,11,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,45,46,47,49,50,107,110,114,115,116,119,120,121,124,125,126,128,131,132) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\wartm\Documents\FHNW\immo_challenge\data\immo_data_202208_v2.csv")


In [22]:
df = df.drop(columns=["Unnamed: 0.1","Unnamed: 0"])

In [23]:
df.head()

,Municipality,Living space,Plot area,Floor space,Availability,location,description,detailed_description,url,table,...,features,description_detailed,Floor space:,Number of floors:,Volume:,plz,Number of toilets:,Gross yield:,Minimum floor space:,space_cleaned
0,Biberstein,100 m²,NaN,NaN,On request,"5023 Biberstein, AG","3.5 rooms, 100 m²«Luxuriöse Attika-Wohnung mit...",DescriptionLuxuriöse Attika-Wohnung direkt an ...,https://www.immoscout24.ch//en/d/penthouse-buy...,b <article class=####Box-cYFBPY hKrxoH####><h2...,...,NaN,NaN,NaN,NaN,NaN,5023.0,NaN,NaN,NaN,NaN
1,Biberstein,156 m²,222 m²,242 m²,On request,"Buhldenstrasse 8d5023 Biberstein, AG","4.5 rooms, 156 m²«Stilvolle Liegenschaft - ruh...",DescriptionStilvolle Liegenschaft an ruhiger L...,https://www.immoscout24.ch//en/d/terrace-house...,b <article class=####Box-cYFBPY hKrxoH####><h2...,...,NaN,NaN,NaN,NaN,NaN,5023.0,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,"5022 Rombach, AG","2.5 rooms, 93 m²«Moderne, lichtdurchflutete At...","detail_responsive#description_title2,5 Zimmerw...",https://www.immoscout24.ch//en/d/penthouse-buy...,b <article class=####Box-cYFBPY hKrxoH####><h2...,...,NaN,NaN,NaN,NaN,NaN,5022.0,NaN,NaN,NaN,NaN
3,Biberstein,154 m²,370 m²,257 m²,On request,"Buhaldenstrasse 8A5023 Biberstein, AG","4.5 rooms, 154 m²«AgentSelly - Luxuriöses Eckh...",DescriptionDieses äusserst grosszügige Minergi...,https://www.immoscout24.ch//en/d/detached-hous...,b <article class=####Box-cYFBPY hKrxoH####><h2...,...,NaN,NaN,NaN,NaN,NaN,5023.0,NaN,NaN,NaN,NaN
4,Küttigen,142 m²,NaN,NaN,On request,"5022 Rombach, AG","4.5 rooms, 142 m²«MIT GARTENSITZPLATZ UND VIEL...",DescriptionAus ehemals zwei Wohnungen wurde ei...,https://www.immoscout24.ch//en/d/flat-buy-romb...,b <article class=####Box-cYFBPY hKrxoH####><h2...,...,NaN,NaN,NaN,NaN,NaN,5022.0,NaN,NaN,NaN,NaN


In [29]:
df.isna().sum()

Municipality              856
Living space            10649
Plot area               17260
Floor space             18853
Availability             9784
                        ...  
plz                        13
Number of toilets:      21393
Gross yield:            21452
Minimum floor space:    21464
space_cleaned           12340
Length: 132, dtype: int64

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22481 entries, 0 to 22480
Columns: 132 entries, Municipality to space_cleaned
dtypes: float64(63), int64(2), object(67)
memory usage: 22.6+ MB


In [25]:
df = df[~df["price_cleaned"].isna()]

In [27]:
list(df.columns)

['Municipality',
 'Living space',
 'Plot area',
 'Floor space',
 'Availability',
 'location',
 'description',
 'detailed_description',
 'url',
 'table',
 'Floor',
 'detail_responsive#municipality',
 'detail_responsive#surface_living',
 'detail_responsive#floor',
 'detail_responsive#available_from',
 'Gemeinde',
 'Wohnfläche',
 'Stockwerk',
 'Nutzfläche',
 'Verfügbarkeit',
 'Grundstücksfläche',
 'detail_responsive#surface_property',
 'detail_responsive#surface_usable',
 'Commune',
 'Surface habitable',
 'Surface du terrain',
 'Surface utile',
 'Disponibilité',
 'Étage',
 'Comune',
 'Superficie abitabile',
 'Disponibilità',
 'Gross return',
 'Piano',
 'Superficie del terreno',
 'Superficie utile',
 'Municipality_merged',
 'Floor_merged',
 'Living_space_merged',
 'Floor_space_merged',
 'Plot_area_merged',
 'Availability_merged',
 'location_parsed',
 'title',
 'details',
 'address',
 'price',
 'link',
 'details_structured',
 'lat',
 'lon',
 'index',
 'ForestDensityL',
 'ForestDensityM',
 '

In [35]:
# Sicherstellen, dass die notwendigen Spalten vorhanden sind
required_columns = ['lat', 'lon', 'price_cleaned']
if not all(column in df.columns for column in required_columns):
    raise ValueError(f"Die folgenden Spalten fehlen im Datensatz: {set(required_columns) - set(df.columns)}")

# Dash-App initialisieren
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Interaktives Immobilien-Dashboard"),

    dcc.Graph(id='map-plot'),

    # Optional: Filter nach Preis hinzufügen
    html.Label("Preisbereich auswählen:"),
    dcc.RangeSlider(
        id='price-slider',
        min=df['price_cleaned'].min(),
        max=df['price_cleaned'].max(),
        value=[df['price_cleaned'].min(), df['price_cleaned'].max()],
        marks={int(price): f"{int(price)} CHF" for price in df['price_cleaned'].quantile([0, 0.25, 0.5, 0.75, 1])}
    )
])

@app.callback(
    Output('map-plot', 'figure'),
    Input('price-slider', 'value')
)
def update_map(price_range):
    # Daten nach ausgewähltem Preis filtern
    filtered_df = df[(df['price_cleaned'] >= price_range[0]) & (df['price_cleaned'] <= price_range[1])]

    # Karte erstellen
    fig = px.scatter_mapbox(
        filtered_df,
        lat="lat",
        lon="lon",
        hover_name="index",
        hover_data={
            "price_cleaned": True,
            "description": True,
            "details": True,
            "lat": False,
            "lon": False
        },
        color="price_cleaned",
        color_continuous_scale=px.colors.cyclical.IceFire,
        size_max=15,
        zoom=6,
        height=600
    )

    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

    return fig

# App im Jupyter Notebook ausführen
app.run_server(mode='inline', debug=True)